In [10]:
import requests
from dotenv import load_dotenv
from langchain_core.tools import tool
import os

load_dotenv()  # Load environment variables from .env


load_dotenv()

@tool
def search_recipes(
    query: str, 
    includeIngredients: str, 
    excludeIngredients: str, 
    diet: str, 
    cuisine: str, 
    addRecipeInformation: bool, 
    number: int,
) -> dict:
    """Search for recipes based on various filters and preferences.

    Args:
        query: Search term for the recipe (e.g., 'pasta').
        includeIngredients: Ingredients that must be included in the recipes (comma-separated).
        excludeIngredients: Ingredients to exclude from the recipes (comma-separated).
        diet: Dietary preference (e.g., 'vegetarian').
        cuisine: Cuisine type (e.g., 'italian').
        addRecipeInformation: Whether to include detailed recipe information (True/False).
        number: Number of results to return.

    Returns:
        A dictionary containing the search results.
    """
    result = {
        "query": query,
        "includeIngredients": includeIngredients,
        "excludeIngredients": excludeIngredients,
        "diet": diet,
        "cuisine": cuisine,
        "addRecipeInformation": addRecipeInformation,
        "number": number
    }
    return result


@tool
def search_ingredient_substitutes(
    ingredient_name: str
) -> dict:
    """Fetches ingredient substitutes from the Spoonacular API.

    Args:
        ingredient_name (str): The name of the ingredient to find substitutes for.

    Returns:
        dict: A dictionary containing the substitutes and relevant information.
    """
    # Return parameters for fetching substitutes
    result = {
        "ingredient_name": ingredient_name
    }
    return result

@tool
def conversation(query: str) -> str:
    """This function simply returns the input query as part of a normal conversation."""
    return query


In [11]:
import requests
from dotenv import load_dotenv
import os

load_dotenv()

def fetch_recipes(parameters):
    # API Key and URL
    API_KEY = os.getenv("SPOONACULAR_API_KEY")
    BASE_URL = "https://api.spoonacular.com/recipes/complexSearch"

    # Parameters for the search
    params = {
        "query": parameters['query'],
        "includeIngredients": parameters['includeIngredients'],
        "excludeIngredients": parameters['excludeIngredients'],
        "diet": parameters['diet'],
        "cuisine": parameters['cuisine'],
        "addRecipeInformation": parameters['addRecipeInformation'],
        "number": parameters['number'],
        "apiKey": API_KEY
    }

    # Make the API request
    response = requests.get(BASE_URL, params=params)

    # Check the response status and return results
    if response.status_code == 200:
        recipes = response.json()
        return recipes
    else:
        return {
            "error": f"Failed to fetch recipes: {response.status_code}",
            "details": response.json()
        }



def fetch_substitutes(parameter: dict) -> dict:
    """
    Makes an API request to fetch substitutes for a given ingredient.

    Args:
        parameter (dict): A dictionary containing 'ingredient_name' and 'api_key'.

    Returns:
        dict: A dictionary containing the substitutes or error details.
    """

    # API base URL
    BASE_URL = "https://api.spoonacular.com/food/ingredients/substitutes"
    API_KEY = os.getenv("SPOONACULAR_API_KEY")

    # Parameters for the request
    params = {
        "ingredientName": parameter["ingredient_name"],
        "apiKey": API_KEY
    }

    try:
        # Make the API request
        response = requests.get(BASE_URL, params=params)

        # Validate response status
        if response.status_code == 200:
            substitutes = response.json()
            return {
                "substitutes": substitutes.get("substitutes", []),
                "message": substitutes.get("message"),
            }
        else:
            return {
                "error": f"Failed to fetch substitutes. Status code: {response.status_code}",
                "details": response.json(),
            }
    except requests.exceptions.RequestException as e:
        # Handle network-related errors
        return {"error": f"An error occurred while making the API request: {str(e)}"}


In [12]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os 
from dotenv import load_dotenv
import os 
from dotenv import load_dotenv

def get_ai_response(input_text: str) -> str:
    """
    Get AI response using LangChain and OpenAI.
    
    Args:
        input_text (str): User input text/question
        
    Returns:
        str: AI generated response
    """
    load_dotenv()

    os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
    os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')

    # Prompt Template
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a friendly and intelligent assistant. You can provide helpful answers, engage in casual conversation, and be thoughtful and patient with your responses. Feel free to ask follow-up questions if needed."),
            ("user", "I need assistance with: {question}")
        ]
    )

    llm = ChatOpenAI(model='gpt-3.5-turbo')
    output_parser = StrOutputParser()
    chain = prompt|llm|output_parser

    if input_text:
        result = chain.invoke({"question": input_text})
        return result
    return ""


In [13]:
import os
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import LLMChain
from langchain.llms import Cohere
from langchain.prompts import PromptTemplate

def get_recipe_details(url):
    """
    Retrieves and analyzes recipe details from a given URL using LangChain and Cohere.
    
    Args:
        url (str): URL of the recipe webpage to analyze
        
    Returns:
        str: Detailed response containing ingredients and instructions
    """
    # Load the webpage content
    loader = WebBaseLoader(url)
    docs = loader.load()

    # Initialize API key (ensure the COHERE_API_KEY environment variable is set)
    os.environ['COHERE_API_KEY'] = os.getenv('COHERE_API_KEY')

    # Split the documents into chunks for processing
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
    documents = text_splitter.split_documents(docs)

    # Initialize Cohere LLM
    llm = Cohere(
        temperature=0.7,
        max_tokens=300  # Adjust the token limit as needed
    )

    prompt = PromptTemplate(
        input_variables=["context", "input"],
        template="""
        You are a highly intelligent and helpful assistant specialized in providing precise and insightful answers based on the given context. Always base your response strictly on the provided information and include step-by-step reasoning to ensure clarity and accuracy. Avoid assumptions and external knowledge unless explicitly asked for.

        Here is the context for your response:
        <context>
        {context}
        </context>
        
        Now, answer the following question:
        Question: {input}
        """
    )


    # Create the LLM chain with the prompt template
    chain = LLMChain(llm=llm, prompt=prompt)

    # Process the documents to get the ingredients and preparation steps
    response = chain.run({"context": "\n".join([doc.page_content for doc in documents]), "input": "Ingredients and Preparation Steps"})

    return response




In [49]:
# from langchain_cohere import ChatCohere
# import os

# load_dotenv()

# os.environ["COHERE_API_KEY"] = os.getenv("COHERE_API_KEY")


# llm = ChatCohere(model="command-r-plus-04-2024")

# tools = [search_recipes, search_ingredient_substitutes]
# llm_with_tools = llm.bind_tools(tools)

# query = "I want a vegetarian recipe that includes mushrooms and spinach, excludes dairy and nuts, is high in protein, and takes less than 30 minutes to prepare. Provide 3 detailed results with nutritional information."

# parameters = llm_with_tools.invoke(query).tool_calls

# tool_name = parameters[0]['name']

# if tool_name == "search_ingredient_substitutes":
#     tool_args = parameters[0]['args']
#     substitutes = fetch_substitutes(tool_args)
#     for substitute in substitutes.get("substitutes", []):
#         print(substitute)
# elif tool_name == "search_recipes":
#     tool_args = parameters[0]['args']
#     recipes = fetch_recipes(tool_args)
#     print(recipes)
#     for recipe in recipes['results']:
#         print(recipe['title'])
#         print(recipe['sourceUrl'])
#         print(recipe['image'])
#         print(get_recipe_details(recipe['sourceUrl']))
#         print("-" * 30)
# else:
#     print("Invalid tool call")



In [50]:
from langchain_cohere import ChatCohere
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
cohere_key = os.getenv("COHERE_API_KEY")
if not cohere_key:
    raise ValueError("COHERE_API_KEY is missing. Please add it to your .env file.")
os.environ["COHERE_API_KEY"] = cohere_key

# Initialize Cohere chat model
llm = ChatCohere(model="command-r-plus-04-2024")

# Define tools
tools = [search_recipes, search_ingredient_substitutes]
llm_with_tools = llm.bind_tools(tools)

# User query
query = "I want a vegetarian recipe that includes mushrooms and spinach, excludes dairy and nuts, is high in protein, and takes less than 30 minutes to prepare. Provide 3 detailed results with nutritional information."

# Invoke tools and handle tool calls
try:
    parameters = llm_with_tools.invoke(query).tool_calls
except Exception as e:
    print(f"Error invoking tools: {str(e)}")
    exit(1)

if not parameters:
    print("No tools were invoked. Please check the query.")
    exit(1)

# Execute the appropriate tool based on the response
tool_name = parameters[0].get('name', None)
tool_args = parameters[0].get('args', {})

if tool_name == "search_ingredient_substitutes":
    result = fetch_substitutes(tool_args)
    if "error" in result:
        print(f"Error fetching substitutes: {result['error']}")
    else:
        print(result)
elif tool_name == "search_recipes":
    recipes = fetch_recipes(tool_args)
    if "error" in recipes:
        print(f"Error fetching recipes: {recipes['error']}")
    else:
        for recipe in recipes.get('results', []):
            print(recipe.get('title', 'No title'))
            print(recipe.get('sourceUrl', 'No URL'))
            print(recipe.get('image', 'No image available'))
            print(get_recipe_details(recipe.get('sourceUrl', '')))
            print("-" * 30)
else:
    print("Invalid tool call")


Mushroom Hummus Crostini
https://www.foodista.com/recipe/DMRJSD86/mushroom-crostini-with-harissa-hummus
https://img.spoonacular.com/recipes/1095745-312x231.jpg
The ingredients for Mushroom Crostini with Harissa Hummus are as follows:

- Crostini: 1 loaf thin, crusty bread (such as a french stick)
- Mushrooms: 500g sliced
- Spinach: 2-3 handfuls
- Hummus: 1 14oz can chickpeas, 1 clove garlic, 1 tbsp lemon juice, 1 tbsp tahini, 1 tbsp olive oil, 2-3 tbsp water, 1/4 tsp ground cumin, 2-3 tbsp harissa paste (adjust depending on how spicy you want the hummus)

The preparation steps are as follows:

1. To make the hummus, place all ingredients in a food processor and blend until smooth. Adjust the amount of water and harissa pasta depending on how thick and spicy you want your hummus. It should still be spreadable. Set the hummus aside.
2. Slice the mushrooms and fry them in a shallow pan until they are golden and crispy. Once they are ready, push them to one side of the pan and add the spin

In [38]:
import os
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents.stuff import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_cohere.llms import Cohere
from langchain_cohere import CohereEmbeddings

def get_recipe_details(url):
    """
    Retrieves and analyzes recipe details from a given URL using LangChain and Cohere.
    
    Args:
        url (str): URL of the recipe webpage to analyze
        
    Returns:
        str: Detailed response containing ingredients and instructions
    """
    # Load the webpage content
    loader = WebBaseLoader(url)
    docs = loader.load()

    # Initialize API key
    os.environ['COHERE_API_KEY'] = os.getenv('COHERE_API_KEY')

    # Split the document into smaller chunks for processing (reduce chunk size for efficiency)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=20)
    documents = text_splitter.split_documents(docs)

    # Limit the number of documents processed to reduce time complexity
    documents = documents[:20]  # Process only the first 20 chunks (adjust as necessary)

    # Create a vector store with FAISS
    db = FAISS.from_documents(documents, CohereEmbeddings(model="embed-english-light-v3.0"))

    # Create the Cohere LLM object (adjust max_tokens to a reasonable limit for performance)
    llm = Cohere(
        temperature=0.7,
        max_tokens=300  # Lower token limit to optimize performance
    )

    # Define the prompt template to extract the ingredients and preparation steps
    prompt = PromptTemplate(
        input_variables=["context", "input"],
        template="""
        You are a highly intelligent and helpful assistant specialized in providing precise and insightful answers based on the given context. Always base your response strictly on the provided information and include step-by-step reasoning to ensure clarity and accuracy. Avoid assumptions and external knowledge unless explicitly asked for.

        Here is the context for your response:
        <context>
        {context}
        </context>
        
        Now, answer the following question:
        Question: {input}
        """
    )

    # Create the document chain for querying
    document_chain = create_stuff_documents_chain(llm, prompt)

    # Create the retriever from the FAISS vector store
    retriever = db.as_retriever()

    # Create the retrieval chain
    retrieval_chain = create_retrieval_chain(retriever, document_chain)

    # Query the chain to retrieve recipe details (ingredients and preparation instructions)
    response = retrieval_chain.invoke({
        "input": "What are the ingredients and the instructions or preparation for this recipe?"
    })

    return response['answer']

x = get_recipe_details("https://www.foodista.com/recipe/DMRJSD86/mushroom-crostini-with-harissa-hummus")
print(x)


The ingredients for Mushroom Crostini with Harissa Hummus are:

- 1 loaf Thin, Crusty Bread such as french stick
- 500 g Mushrooms sliced
- 2-3 handfuls Fresh Spinach
- 1 14oz can chickpeas
- 1 clove garlic
- 1 tbsp Lemon Juice
- 1 tbsp Tahini
- 1 tbsp olive oil
- 2-3 tbsp Water
- 1/4 tsp Ground Cumin
- 2-3 tbsp Harissa Paste adjust depending on how spicy you want the hummus

The instructions for preparation are as follows:

1. To make the hummus, place all ingredients in a food processor and blend until smooth. Adjust the amount of water and harissa pasta depending on how thick and spicy you want your hummus. It should still be spreadable. Set the hummus aside.
2. Slice the mushrooms and fry in a shallow pan until they are golden and crispy. Once they are ready, push them to one side of the pan and add the spinach to wilt.
3. To assemble the crostini, spread some hummus on a slice of the bread (you can toast it if you like), then top with some spinach and mushrooms. Serve immediately.

In [47]:
import os
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import LLMChain
from langchain.llms import Cohere
from langchain.prompts import PromptTemplate

def get_recipe_details(url):
    """
    Retrieves and analyzes recipe details from a given URL using LangChain and Cohere.
    
    Args:
        url (str): URL of the recipe webpage to analyze
        
    Returns:
        str: Detailed response containing ingredients and instructions
    """
    # Load the webpage content
    loader = WebBaseLoader(url)
    docs = loader.load()

    # Initialize API key (ensure the COHERE_API_KEY environment variable is set)
    os.environ['COHERE_API_KEY'] = os.getenv('COHERE_API_KEY')

    # Split the documents into chunks for processing
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
    documents = text_splitter.split_documents(docs)

    # Initialize Cohere LLM
    llm = Cohere(
        temperature=0.7,
        max_tokens=300  # Adjust the token limit as needed
    )

    # Define the prompt template for extracting ingredients and preparation
    # prompt = PromptTemplate(
    #     input_variables=["context", "input"],
    #     template="""
    #     You are an intelligent assistant that can analyze recipe content. Your task is to extract the ingredients and the preparation steps for the given recipe from the provided context.

    #     Here is the context:
    #     <context>
    #     {context}
    #     </context>

    #     Now, please extract the ingredients and preparation steps. Provide the ingredients in a bullet-point format and the preparation steps in a numbered list.
    #     """
    # )
    prompt = PromptTemplate(
        input_variables=["context", "input"],
        template="""
        You are a highly intelligent and helpful assistant specialized in providing precise and insightful answers based on the given context. Always base your response strictly on the provided information and include step-by-step reasoning to ensure clarity and accuracy. Avoid assumptions and external knowledge unless explicitly asked for.

        Here is the context for your response:
        <context>
        {context}
        </context>
        
        Now, answer the following question:
        Question: {input}
        """
    )


    # Create the LLM chain with the prompt template
    chain = LLMChain(llm=llm, prompt=prompt)

    # Process the documents to get the ingredients and preparation steps
    response = chain.run({"context": "\n".join([doc.page_content for doc in documents]), "input": "Ingredients and Preparation Steps"})

    return response

# x = get_recipe_details("https://www.foodista.com/recipe/BYJ4Q2M5/miso-soup-with-thin-noodles")
# print(x)



The ingredients for the miso soup with thin noodles are laid out below:

- 2 cups of water
- A bunch of fresh spinach
- 1/2 of a block of firm tofu
- 8 shitaki mushrooms, cut up lengthwise
- 2 small chives, cut into squares
- 1 yellow onion, chopped up
- 1/2 package of a  of Thai Kitchen Thin Rice Noodles
- 1 ginger
- 1 parsnip, cut up
- 6 baby carrots, cut up
- 1 zucchini, cut up
- A pinch of red pepper flakes
- A pinch of ginger powder

Below are the steps on how to prepare the miso soup with thin noodles:

1. After the miso has been prepared, start adding the "stuff" to the soup pot. It can be your preference, but I opted to start with the onions and chives and then added the zucchini, parsnip, carrots, mushrooms, and ginger. 
2. Cover the pot and let cook on a low flame for 20-30 minutes, tasting as you go. 
3. Add the tofu and pasta, allowing the pasta to cook for 8-10 minutes. Taste the soup, adding red pepper and turn off the flame when ready. 
4. Place spinach on the bottom of 

In [ ]:
# https://www.foodista.com/recipe/BYJ4Q2M5/miso-soup-with-thin-noodles





In [ ]:
# The ingredients for Mushroom Crostini with Harissa Hummus are as follows:

# - Crostini: 1 loaf thin, crusty bread (such as French stick)
# - Mushrooms: 500g sliced
# - Spinach: 2-3 handfuls
# - Hummus: 1 14oz can chickpeas, 1 clove garlic, 1 tbsp lemon juice, 1 tbsp tahini, 1 tbsp olive oil, 2-3 tbsp water, 1/4 tsp ground cumin, 2-3 tbsp harissa paste (adjust amounts depending on spiciness)

# The preparation steps are as follows:

# 1. To make the hummus, place all ingredients in a food processor and blend until smooth. Adjust the amount of water and harissa pasta depending on how thick and spicy you want your hummus. Set the hummus aside.
# 2. Slice the mushrooms and fry them in a shallow pan until they are golden and crispy. Once they are ready, push them to one side of the pan and add the spinach to wilt.
# 3. To assemble the crostini, spread some hummus on a slice of the bread (you can toast it if you like), then top with some spinach and mushrooms. Serve immediately. 

# Please note that the recipe serves 4 and can be adjusted according to your preferences. 

In [9]:
from langchain_cohere import ChatCohere
import os
from dotenv import load_dotenv



# Load environment variables
load_dotenv()


cohere_key = os.getenv("COHERE_API_KEY")
if not cohere_key:
    raise ValueError("COHERE_API_KEY is missing. Please add it to your .env file.")
os.environ["COHERE_API_KEY"] = cohere_key

# Initialize Cohere chat model
llm = ChatCohere(model="command-r-plus-04-2024")

# Define tools
tools = [search_recipes, search_ingredient_substitutes]
llm_with_tools = llm.bind_tools(tools)

# User query
query = "I want substitutes for butter"

# Invoke tools and handle tool calls
parameters = llm_with_tools.invoke(query).tool_calls


# Execute the appropriate tool based on the response
tool_name = parameters[0].get('name', None)
tool_args = parameters[0].get('args', {})

if tool_name == "search_ingredient_substitutes":
    result = fetch_substitutes(tool_args)
    if "error" in result:
        print(f"Error fetching substitutes: {result['error']}")
    else:
        for substitute in result['substitutes']:
            print(substitute)
        # bot_response = result
elif tool_name == "search_recipes":
    recipes = fetch_recipes(tool_args)
    if "error" in recipes:
        print(f"Error fetching recipes: {recipes['error']}")
    else:
        for recipe in recipes.get('results', []):
            print(recipe.get('title', 'No title'))
            print(recipe.get('sourceUrl', 'No URL'))
            print(recipe.get('image', 'No image available'))
            result = get_recipe_details(recipe.get('sourceUrl', ''))
            print(result)

            # bot_response = result
else:
    print("Invalid tool call")


1 cup = 7/8 cup shortening and 1/2 tsp salt
1 cup = 7/8 cup vegetable oil + 1/2 tsp salt
1/2 cup = 1/4 cup buttermilk + 1/4 cup unsweetened applesauce
1 cup = 1 cup margarine
